In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

driver = webdriver.Chrome()
driver.maximize_window()

wait = WebDriverWait(driver, 8)

def wait_for_page_to_load(driver, wait):
    page_title = driver.title
    try:
        wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
    except:
        print("The page did not get fully loaded")
    else:
        print("The page is fully loaded")

url = "https://quotes.toscrape.com/scroll"
driver.get(url)
wait_for_page_to_load(driver, wait)

data = []
page_count = 0

while True:
    page_count += 1
    try:
        rows = driver.find_elements(By.CLASS_NAME, 'quote')
        for row in rows:
            try:
                quotes = row.find_element(By.CLASS_NAME, 'text').text
            except:
                print("Quotes not extracted")
                quotes = ""

            try:
                author_name = row.find_element(By.CLASS_NAME, 'author').text
            except:
                print("Author not extracted")
                author_name = ""

            try:
                tag_name = row.find_element(By.CLASS_NAME, 'tags').text
            except:
                print("Tags not extracted")
                tag_name = ""

            all_values = {
                "quotes": quotes,
                "author_name": author_name,
                "tag_name": tag_name
            }
            data.append(all_values)

        # check if Next Page button exists
        try:

        # infinite scrolling
            prev_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == prev_height:
                    break
                prev_height = new_height

        except:
            print(f"No more pages after page {page_count}.")
            break

    except:
        print("Something went wrong in scraping")
        break

df = pd.DataFrame(data)
print(df)
